# Predict Traffic Accident Severity


#### Priyanka Dave
#### October 2020

<img src="img.jpg" alt="Predict Traffic Accident Severity" title="Predict Traffic Accident Severity" />

## Table of content
1. [Introduction](#intro)
2. [Data](#data)

##  1. Introduction<a id='intro'>

Road traffic injuries are currently estimated to be the eighth leading cause of death across all age groups globally, and are predicted to become the seventh leading cause of death by 2030.

In an effort to reduce the frequency of car collisions in a community, an algorithim must be developed to predict the severity of an accident given the current weather, road and visibility conditions. When conditions are bad, this model will alert drivers to remind them to be more careful.

These insights, could allow law enforcement bodies to allocate their resources more effectively in advance of potential accidents, preventing when and where a severe accidents can occur as well as saving both, time and money. In addition, this knowledge of a severe accident situation can be warned to drivers so that they would drive more carefully or even change their route if it is possible or to hospital which could have set everything ready for a severe intervention in advance.

Governments should be highly interested in accurate predictions of the severity of an accident, in order to reduce the time of arrival and thus save a significant amount of people each year. Others interested could be private companies investing in technologies aiming to improve road safeness.

## 2. Data<a id='data'>

- The original data for this project comes from the following [Click Here](https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv).
- Initial dataset having 194673 samples and 38 features.
- After removing features having identification numbers, features having more than 40% null values and features which are irrelevant to the application, I left with below list of features:

    1. **SEVERITYCODE** - Target variable. 1: Property Damege, 2: Injury
    1. **ADDRTYPE** - Collision address type. Alley, Block, Intersection
    1. **COLLISIONTYPE** - Collision type. Parked Car, Angles, Rear Ended, etc...
    1. **PERSONCOUNT** - Total number of people involved in the collision.
    1. **VEHCOUNT** - Total number of vehicles involved in the collision.
    1. **JUNCTIONTYPE** - Category of junction at which collision took place
    1. **UNDERINFL** - Whether or not a driver involved was under the influence of drugs or alcohol.
    1. **WEATHER** - Description of the weather conditions during the time of the collision.
    1. **ROADCOND** - Condition of the road during the collision.
    1. **LIGHTCOND** - Light conditions during the collision.
    1. **MONTH** - Month in which collision happened.
    1. **DAY_OF_WEEK** - Day of week of collision.


